# Diversity measures for Data Valuation

As the diversity measures concerns the whole dataset and are quite time consuming to compute, here we explore different diversity measures.

# 1. Setup

In [1]:
#First, we define Alice's model M. We assume a simple CNN model.
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
import matplotlib.pyplot as plt
import os

#Don't use GPU for now
os.environ["CUDA_VISIBLE_DEVICES"] = ""

class LeNet(nn.Sequential):
    """
    Adaptation of LeNet that uses ReLU activations
    """

    # network architecture:
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.act = nn.Softmax(dim=1)
        

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.act(x)
        return x
    
model = LeNet()

os.makedirs('data', exist_ok=True)
torch.save(model.state_dict(), 'data/model.pth')
torch.save(model, 'data/alice_model.pth')

#Next, we define the data loader for CIFAR-10 dataset.
import torchvision
import random
import torchvision.transforms as transforms
import numpy as np

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=False,transform=transform,download=True)


# Randomly select 100 images as Bob's dataset
indices = random.sample(range(len(trainset)), 100)
bob_images = np.array([trainset[i][0].numpy() for i in indices])
bob_labels = np.array([trainset[i][1]  for i in indices])

#Randomly select 500 images as Alice's trained dataset
indices = random.sample(range(len(trainset)), 500)
alice_images = np.array([trainset[i][0].numpy() for i in indices])
alice_labels = np.array([trainset[i][1]  for i in indices])

Files already downloaded and verified


# 2. Dimension reduction

As the diversity measures are time consuming to compute, we will first reduce the dimensionality of the dataset using random projection.

In [2]:
import hashlib

#Secure random sharing
bob_chosen_number = random.getrandbits(32)
bob_nonce = random.getrandbits(32)
bob_hashed_number = hashlib.sha256(f"{bob_chosen_number}{bob_nonce}".encode()).hexdigest()
#Bob sends alice this hash
print("Alice receives Bob's hash: ", bob_hashed_number)

alice_chosen_number = random.getrandbits(32)
alice_nonce = random.getrandbits(32)
alice_hashed_number = hashlib.sha256(f"{alice_chosen_number}{alice_nonce}".encode()).hexdigest()
#Alice sends bob this hash
print("Bob receives Alice's hash: ", alice_hashed_number)

#Alice sends Bob her chosen number and nonce
print("Bob receives Alice's chosen number and nonce: ", alice_chosen_number, alice_nonce)
#Bob verifies the hash
assert hashlib.sha256(f"{alice_chosen_number}{alice_nonce}".encode()).hexdigest() == alice_hashed_number
#Bob sends Alice his chosen number and nonceq
print("Alice receives Bob's chosen number and nonce: ", bob_chosen_number, bob_nonce)
#Alice verifies the hash
assert hashlib.sha256(f"{bob_chosen_number}{bob_nonce}".encode()).hexdigest() == bob_hashed_number

#Calculate common random number
common_random_number = alice_chosen_number ^ bob_chosen_number
print("Common random number: ", common_random_number)

Alice receives Bob's hash:  52b044644b5a707f4f85a1aeda79e2aae43e3250f5191c42d27ddec3bf550cc9
Bob receives Alice's hash:  46b75d97deca09b79107b5a1fcda02af3e20089d6e65799fd4807fa90ea6a72e
Bob receives Alice's chosen number and nonce:  1682463095 1094446282
Alice receives Bob's chosen number and nonce:  1312748263 3460261008
Common random number:  712420240


In [3]:
#In reality, Bob and Alice will independently do this on their own machines, but they have the same common random number.
#In the malicious case, this will get a ZKP to prove that they correctly manipulated the dataset.
from sklearn.random_projection import SparseRandomProjection

# Flatten the images
bob_images_flat = bob_images.reshape(bob_images.shape[0], -1)
alice_images_flat = alice_images.reshape(alice_images.shape[0], -1)

# Define the random projection transformer
n_components = 100  # You can adjust this number based on your needs
alice_transformer = SparseRandomProjection(n_components=n_components, random_state = common_random_number)
bob_transformer = SparseRandomProjection(n_components=n_components, random_state = common_random_number)

# Fit and transform the images
bob_images_reduced = alice_transformer.fit_transform(bob_images_flat)
alice_images_reduced = bob_transformer.fit_transform(alice_images_flat)

print("Bob's images reduced shape:", bob_images_reduced.shape)
print("Alice's images reduced shape:", alice_images_reduced.shape)

Bob's images reduced shape: (100, 100)
Alice's images reduced shape: (500, 100)


# 2. Alice's clustering

Alice first performs clustering on her dataset to identify the cluster centers. This is done locally on Alice's side, and we do not protect this operation.

In [4]:
#First, we run the Kmeans clustering algorithm locally on Bob's device
from sklearn.cluster import KMeans

# Set the number of clusters
K = 10

# Perform K-means clustering
kmeans = KMeans(n_clusters=K).fit(alice_images_reduced)

# Get the cluster labels
cluster_labels = kmeans.labels_

# Get the cluster centers
cluster_centers = kmeans.cluster_centers_

print("Cluster centers shape:", cluster_centers.shape)

Cluster centers shape: (10, 100)


# 3. Find points furthest to cluster centers

The points that are furthest to cluster centeres are selected as the points that are least similar to the rest of Alice's dataset.

In [5]:
# Plaintext computation
distances = np.linalg.norm(bob_images_reduced[:,np.newaxis,:]- cluster_centers, axis=2)
print("Distances shape:", distances.shape)
min_distances = np.min(distances, axis=1)
print("Min distances shape:", min_distances.shape)
sorted_indices = np.argsort(-min_distances)
sorted_points = bob_images_reduced[sorted_indices]
sorted_distances = min_distances[sorted_indices]
print(sorted_distances[:2], sorted_indices[:2])

#We can now use the indices to get the corresponding unreduced images and labels

Distances shape: (100, 10)
Min distances shape: (100,)
[34.600197 31.051662] [38 57]


In [6]:
#Semi-honest MPC computation
import crypten
import crypten.mpc as mpc
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "" 
crypten.init()
torch.set_num_threads(1)

BOB_INPUT_PATH = 'data/bob_images_reduced.pth'
ALICE_INPUT_PATH = 'data/alice_cluster_centers.pth'
OUTPUT_PATH = 'data/encrypted_min_distances.pth'

#First we save Bob and Alice's inputs to a file
os.makedirs('data', exist_ok=True)
torch.save(torch.tensor(bob_images_reduced), BOB_INPUT_PATH)
torch.save(torch.tensor(cluster_centers), ALICE_INPUT_PATH)

ALICE = 0
BOB = 1

@mpc.run_multiprocess(world_size=2)
def secure_sort_indices():
    # Load the inputs
    bob_images_reduced = crypten.load_from_party(BOB_INPUT_PATH,src=BOB)
    cluster_centers = crypten.load_from_party(ALICE_INPUT_PATH,src=ALICE)

    # Compute the distances
    diff = (bob_images_reduced.unsqueeze(1) - cluster_centers)
    diff = diff * diff
    pairwise_distances = diff.sum(dim=2)
    encrypted_min_distances, _ = pairwise_distances.min(dim=1)
    crypten.save_from_party(encrypted_min_distances.get_plain_text(),OUTPUT_PATH,src=BOB)

import time
start = time.time()
secure_sort_indices()
end = time.time()
print("Time taken: ", end-start)

/home/thomas/anaconda3/envs/sdv2/lib/python3.11/site-packages/crypten/__init__.py:334: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  result = load_closure(f, **kwargs)
/home

Time taken:  0.24306106567382812


In [7]:
min_distances = torch.load(OUTPUT_PATH)
sorted_indices = np.argsort(-min_distances)
sorted_distances = min_distances.sqrt()[sorted_indices]
print(sorted_distances[:2], sorted_indices[:2])

tensor([34.6002, 31.0516]) tensor([38, 57])


/tmp/ipykernel_869227/1154103820.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  min_distances = torch.load(OUTPUT_PATH)


In [8]:
#Malicious scenario

#Prepare Alice and Bob's private input for MPC
Bob_input = bob_images_reduced
Alice_input = cluster_centers
if not os.path.exists('../MP-SPDZ/Player-Data'):
    os.makedirs('../MP-SPDZ/Player-Data')
p0_path = os.path.join('../MP-SPDZ/Player-Data','Input-P0-0')
p1_path = os.path.join('../MP-SPDZ/Player-Data','Input-P1-0')

#Bob as P0
points_1d = np.array(bob_images_reduced).reshape(-1).tolist()
with open(p0_path, 'w') as f:
    f.write(' '.join(map(lambda x : f"{x:.6f}", points_1d)))
    f.write('\n')

#Alice as P1
points_1d = np.array(cluster_centers).reshape(-1).tolist()
with open(p1_path, 'w') as f:
    f.write(' '.join(map(lambda x : f"{x:.6f}", points_1d)))
    f.write('\n')

In [9]:
#The code for valuation is prepared in ../../MP-SPDZ/Programs/Source/diversity.mpc
# Here we compile the MPC code 
! cd ../MP-SPDZ && ./compile.py diversity

Default bit length for compilation: 64
Default security parameter for compilation: 40
Compiling file Programs/Source/diversity.mpc
Writing to Programs/Bytecode/diversity-TruncPr(7)_47_16-1.bc
Writing to Programs/Bytecode/diversity-TruncPr(6)_47_16-3.bc
Writing to Programs/Bytecode/diversity-LTZ(7)_32-4.bc
Writing to Programs/Bytecode/diversity-LTZ(2)_32-6.bc
Writing to Programs/Schedules/diversity.sch
Writing to Programs/Bytecode/diversity-0.bc
Hash: 1763f98b7cdcb6a95f0a9e1f0510a42b4ebc3369fa3a7a4f138beac00fb0521d
Program requires at most:
        1000 integer inputs from player 1
      150400 integer triples
      151800 integer bits
        2000 integer opens
       10000 integer inputs from player 0
        1521 virtual machine rounds


In [11]:
#MPC for squared loss
import time

start = time.time()
! cd ../MP-SPDZ/ && Scripts/lowgear.sh diversity
end = time.time()
print(f"Time taken for squared loss computation: {end-start}")

Running /home/thomas/secure-data-valuation/MP-SPDZ/Scripts/../lowgear-party.x 0 diversity -pn 16844 -h localhost -N 2
Running /home/thomas/secure-data-valuation/MP-SPDZ/Scripts/../lowgear-party.x 1 diversity -pn 16844 -h localhost -N 2
Using statistical security parameter 40


Min distances: [468.788, 508.324, 598.615, 175.581, 614.495, 282.167, 634.27, 651.802, 532.918, 566.764, 659.043, 415.432, 442.19, 612.057, 551.429, 327.418, 772.345, 786.185, 183.766, 648.102, 310.397, 528.276, 281.315, 609.648, 432.56, 402.932, 513.721, 577.32, 595.087, 343.058, 181.856, 840.897, 426.529, 810.833, 443.896, 219.492, 295.777, 403.601, 1197.17, 372.093, 509.426, 453.936, 395.151, 603.739, 215.599, 356.043, 406.673, 303.254, 413.223, 180.761, 808.18, 85.965, 637.924, 522.085, 225.278, 324.277, 238.695, 964.206, 506.641, 401.538, 325.276, 420.943, 573.015, 290.511, 480.981, 564.827, 684.44, 204.233, 425.748, 711.455, 618.174, 758.005, 276.754, 383.579, 446.727, 749.341, 540.616, 597.641, 667.409, 382.551, 562.054, 454.749, 178.692, 462.063, 713.623, 332.942, 392.233, 251.101, 225.227, 695.766, 626.915, 507.858, 172.273, 477.466, 329.933, 298.135, 312.827, 120.935, 206.916, 482.484]
Significant amount of unused triples of SPDZ gfp distorting the benchmark. This protocol ha

In [13]:
answer = [468.788, 508.324, 598.615, 175.581, 614.495, 282.167, 634.27, 651.802, 532.918, 566.764, 659.043, 415.432, 442.19, 612.057, 551.429, 327.418, 772.345, 786.185, 183.766, 648.102, 310.397, 528.276, 281.315, 609.648, 432.56, 402.932, 513.721, 577.32, 595.087, 343.058, 181.856, 840.897, 426.529, 810.833, 443.896, 219.492, 295.777, 403.601, 1197.17, 372.093, 509.426, 453.936, 395.151, 603.739, 215.599, 356.043, 406.673, 303.254, 413.223, 180.761, 808.18, 85.965, 637.924, 522.085, 225.278, 324.277, 238.695, 964.206, 506.641, 401.538, 325.276, 420.943, 573.015, 290.511, 480.981, 564.827, 684.44, 204.233, 425.748, 711.455, 618.174, 758.005, 276.754, 383.579, 446.727, 749.341, 540.616, 597.641, 667.409, 382.551, 562.054, 454.749, 178.692, 462.063, 713.623, 332.942, 392.233, 251.101, 225.227, 695.766, 626.915, 507.858, 172.273, 477.466, 329.933, 298.135, 312.827, 120.935, 206.916, 482.484]
min_distances = torch.tensor(answer)
sorted_indices = np.argsort(-min_distances)
sorted_distances = min_distances.sqrt()[sorted_indices]
print(sorted_distances[:2], sorted_indices[:2])

tensor([34.6001, 31.0517]) tensor([38, 57])
